In [ ]:
!lscpu

In [ ]:
!uname -a && cat /etc/*release

Linux 8abd1bed502a 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [ ]:
!pwd
!ls -la

/content
total 32
drwxr-xr-x 1 root root  4096 Nov 23 10:24 .
drwxr-xr-x 1 root root  4096 Nov 23 10:23 ..
drwxr-xr-x 4 root root  4096 Nov 21 14:25 .config
-rw-r--r-- 1 root root 14196 Nov 23 10:24 devicequery.zip
drwxr-xr-x 1 root root  4096 Nov 21 14:25 sample_data


In [ ]:
!nvidia-smi

Sat Nov 23 10:24:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!unzip devicequery.zip

Archive:  devicequery.zip
  inflating: devicequery.cu          
  inflating: helper_cuda.h           
  inflating: helper_string.h         


In [ ]:
!nvcc devicequery.cu -o devicequery

In [ ]:
!/content/devicequery

/content/devicequery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount 

## Parallelized version with usage of Shared Memory

In [85]:
code=r'''#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#ifdef __NVCC__
#include <cuda.h>
#endif
#include <sys/time.h>

// Simple define to index into a 1D array from 2D space
#define I2D(num, c, r) ((r)*(num)+(c))

/*
 * `step_kernel_mod` is currently a direct copy of the CPU reference solution
 * `step_kernel_ref` below. Accelerate it to run as a CUDA kernel.
 */

#ifndef BLOCK_SIZE
#define BLOCK_SIZE 16
#endif
#define BLOCK_WIDTH BLOCK_SIZE
#define BLOCK_HEIGHT BLOCK_SIZE
#ifndef SIZE
#define SIZE 1000
#endif
#ifndef STEPS
#define STEPS 200
#endif

#ifdef __NVCC__
#ifndef SHARED
__global__ void step_kernel_mod(int ni, int nj, float fact, float* temp_in, float* temp_out)
{

  /*
    COMMENTS: Remember about coalescence (cudaMallocPitch())
    remeber balancing between L1 and Shared Mem cudaDeviceSetCacheConfig
    __shared__ to access shared memory
    __constant__ to set something on constant memory
  */
  //int i00, im10, ip10, i0m1, i0p1;
  float d2tdx2, d2tdy2;

  int x = threadIdx.x + blockIdx.x * blockDim.x + 1;
  int y = threadIdx.y + blockIdx.y * blockDim.y + 1;

  if(x >= ni-1 || y >= nj-1) return;

  // evaluate derivatives

  d2tdx2 = temp_in[I2D(ni, x - 1, y)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x + 1, y)];
  d2tdy2 = temp_in[I2D(ni, x, y - 1)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x, y + 1)];

  // update temperatures
  temp_out[I2D(ni, x, y)] = temp_in[I2D(ni, x, y)] /*sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)] */ + fact * (d2tdx2 + d2tdy2);

}
#else
__global__ void step_kernel_mod(int ni, int nj, float fact, float* temp_in, float* temp_out)
{

  /*
    COMMENTS: Remember about coalescence (cudaMallocPitch())
    remeber balancing between L1 and Shared Mem cudaDeviceSetCacheConfig
    __shared__ to access shared memory
    __constant__ to set something on constant memory
  */
  //int i00, im10, ip10, i0m1, i0p1;
  float d2tdx2, d2tdy2;

  __shared__ float sharedMem[(BLOCK_WIDTH + 2) * (BLOCK_HEIGHT + 2)];

  int x = threadIdx.x + blockIdx.x * blockDim.x + 1;
  int y = threadIdx.y + blockIdx.y * blockDim.y + 1;

  if(x >= ni-1 || y >= nj-1) return;

  sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)] = temp_in[I2D(ni, x, y)];

// TODO: put most of this inside the same warp

  if(threadIdx.x == 0) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x, threadIdx.y + 1)] = temp_in[I2D(ni, x - 1, y)];
  else if(threadIdx.x == blockDim.x - 1 || x == ni - 2) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 2, threadIdx.y + 1)] = temp_in[I2D(ni, x + 1, y)];

  if(threadIdx.y == 0) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y)] = temp_in[I2D(ni, x, y - 1)];
  else if(threadIdx.y == blockDim.y - 1 || y == nj - 2) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 2)] = temp_in[I2D(ni, x, y + 1)];

  __syncthreads();

  // evaluate derivatives
  d2tdx2 = sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x, threadIdx.y + 1)]
    - 2 * sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)]
    + sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 2, threadIdx.y + 1)];

  d2tdy2 = sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y)]
    - 2 * sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)]
    + sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 2)];

    //d2tdx2 = temp_in[I2D(ni, x - 1, y)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x + 1, y)];
    //d2tdy2 = temp_in[I2D(ni, x, y - 1)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x, y + 1)];

  // update temperatures
  temp_out[I2D(ni, x, y)] = sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)] + fact * (d2tdx2 + d2tdy2);

}
#endif
#endif

void step_kernel_ref(int ni, int nj, float fact, float* temp_in, float* temp_out)
{
  int i00, im10, ip10, i0m1, i0p1;
  float d2tdx2, d2tdy2;


  // loop over all points in domain (except boundary)
  for ( int j=1; j < nj-1; j++ ) {
    for ( int i=1; i < ni-1; i++ ) {
      // find indices into linear memory
      // for central point and neighbours
      i00 = I2D(ni, i, j);
      im10 = I2D(ni, i-1, j);
      ip10 = I2D(ni, i+1, j);
      i0m1 = I2D(ni, i, j-1);
      i0p1 = I2D(ni, i, j+1);

      // evaluate derivatives
      d2tdx2 = temp_in[im10]-2*temp_in[i00]+temp_in[ip10];
      d2tdy2 = temp_in[i0m1]-2*temp_in[i00]+temp_in[i0p1];

      // update temperatures
      temp_out[i00] = temp_in[i00]+fact*(d2tdx2 + d2tdy2);
    }
  }
}

int main()
{
  struct timeval stop, start;
  int istep;
  int nstep = STEPS; // number of time steps

  // Specify our 2D dimensions
  const int ni = SIZE;
  const int nj = SIZE;
  float tfac = 8.418e-5; // thermal diffusivity of silver

  float *temp1_ref, *temp2_ref, *temp1, *temp2, *temp_tmp;

  const int size = ni * nj * sizeof(float);

  temp1_ref = (float*)malloc(size);
  temp2_ref = (float*)malloc(size);
  temp1 = (float*)malloc(size);
  temp2 = (float*)malloc(size);

  // Initialize with random data
  for( int i = 0; i < ni*nj; ++i) {
    temp1_ref[i] = temp2_ref[i] = temp1[i] = temp2[i] = (float)rand()/(float)(RAND_MAX/100.0f);
  }

  gettimeofday(&start, NULL);

#ifndef __NVCC__

  // Execute the CPU-only reference version
  for (istep=0; istep < nstep; istep++) {
    step_kernel_ref(ni, nj, tfac, temp1_ref, temp2_ref);

    // swap the temperature pointers
    temp_tmp = temp1_ref;
    temp1_ref = temp2_ref;
    temp2_ref= temp_tmp;
  }

#else

  float *temp1_dev, *temp2_dev;

  cudaMalloc((void**) &temp1_dev, size);
  cudaMalloc((void**) &temp2_dev, size);

  cudaMemcpy(temp1_dev, temp1, size, cudaMemcpyHostToDevice);
  cudaMemcpy(temp2_dev, temp2, size, cudaMemcpyHostToDevice);

  dim3 block_size(BLOCK_WIDTH, BLOCK_HEIGHT);
  dim3 grid_size = dim3((nj - 3) / BLOCK_WIDTH + 1, (ni - 3) / BLOCK_HEIGHT + 1);

  // Execute the modified version using same data
  for (istep=0; istep < nstep; istep++) {
    step_kernel_mod<<<grid_size, block_size>>>(ni, nj, tfac, temp1_dev, temp2_dev);

    // swap the temperature pointers
    temp_tmp = temp1_dev;
    temp1_dev = temp2_dev;
    temp2_dev = temp_tmp;
  }
  cudaMemcpy(temp1, temp1_dev, size, cudaMemcpyDeviceToHost);

  cudaDeviceSynchronize();

#endif

  gettimeofday(&stop, NULL);
  printf("took %lf s\n", ((stop.tv_sec - start.tv_sec) * 1000000 + stop.tv_usec - start.tv_usec)/1000000.0);

  // float maxError = 0;
  // // Output should always be stored in the temp1 and temp1_ref at this point
  // for( int i = 0; i < ni*nj; ++i ) {
  //   if (abs(temp1[i]-temp1_ref[i]) > maxError) { maxError = abs(temp1[i]-temp1_ref[i]); }
  // }
  //
  // // Check and see if our maxError is greater than an error bound
  // if (maxError > 0.0005f)
  //   printf("Problem! The Max Error of %.5f is NOT within acceptable bounds.\\n", maxError);
  // else
  //   printf("The Max Error of %.5f is within acceptable bounds.\\n", maxError);

  free( temp1_ref );
  free( temp2_ref );
  free( temp1 );
  free( temp2 );

#ifdef __NVCC__
  cudaFree(tmp1_dev);
  cudaFree(tmp2_dev);
#endif

  return 0;
}

'''
with open('assignment-less-dumb.c', 'w') as f:
  f.write(code)


In [78]:
!gcc -O3 -ffast-math assignment-less-dumb.c -o assignment-less-dumb -DSTEPS=200 -DSIZE=13000 && /content/assignment-less-dumb

took 31.249924 s


In [86]:
!nvcc assignment-less-dumb.cu -o assignment-less-dumb -DSTEPS=200 -DSIZE=13000 -DBLOCK_SIZE=16 && /content/assignment-less-dumb

took 2.157767 s


In [87]:
!nvcc assignment-less-dumb.cu -o assignment-less-dumb -DSTEPS=5000 -DSIZE=10000 -DBLOCK_SIZE=16 && /content/assignment-less-dumb

took 21.750136 s


In [62]:
!nvcc assignment-less-dumb.cu -o assignment-less-dumb -DSTEPS=5000 -DSHARED -DSIZE=10000 -DBLOCK_SIZE=16 && /content/assignment-less-dumb

took 32.240333 s


In [98]:
from os import system
from subprocess import check_output

for i in range(6):
  SIZE=3000
  #SIZE=100
  block_size=2**i
  grid_size=(SIZE-3)//block_size+1
  #print(f"{i=} {block_size=}")
  print(f"{i=} <<({grid_size},{grid_size}),({block_size},{block_size})>>")
  out=check_output(f"nvcc assignment-less-dumb.cu -o assignment-less-dumb -DSTEPS=5000 -DSHARED -DSIZE={SIZE} -DBLOCK_SIZE={block_size} && /content/assignment-less-dumb",shell=True,text=True)
  print(out)

i=0 <<(2998,2998),(1,1)>>
took 62.646371 s

i=1 <<(1499,1499),(2,2)>>
took 24.665744 s

i=2 <<(750,750),(4,4)>>
took 7.775655 s

i=3 <<(375,375),(8,8)>>
took 3.337811 s

i=4 <<(188,188),(16,16)>>
took 3.193612 s

i=5 <<(94,94),(32,32)>>
took 2.973721 s



In [82]:
code=r'''#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#ifdef __NVCC__
#include <cuda.h>
#endif
#include <sys/time.h>

// Simple define to index into a 1D array from 2D space
#define I2D(num, c, r) ((r)*(num)+(c))

/*
 * `step_kernel_mod` is currently a direct copy of the CPU reference solution
 * `step_kernel_ref` below. Accelerate it to run as a CUDA kernel.
 */

#ifndef BLOCK_SIZE
#define BLOCK_SIZE 16
#endif
#define BLOCK_WIDTH BLOCK_SIZE
#define BLOCK_HEIGHT BLOCK_SIZE
#ifndef SIZE
#define SIZE 1000
#endif
#ifndef STEPS
#define STEPS 200
#endif

#ifdef __NVCC__
#ifndef SHARED
#ifdef PITCH
__global__ void step_kernel_mod(int ni, int nj, float fact, float* temp_in, float* temp_out, size_t pitch1, size_t pitch2)
#else
__global__ void step_kernel_mod(int ni, int nj, float fact, float* temp_in, float* temp_out)
#endif
{

  /*
    COMMENTS: Remember about coalescence (cudaMallocPitch())
    remeber balancing between L1 and Shared Mem cudaDeviceSetCacheConfig
    __shared__ to access shared memory
    __constant__ to set something on constant memory
  */
  //int i00, im10, ip10, i0m1, i0p1;
  float d2tdx2, d2tdy2;

  int x = threadIdx.x + blockIdx.x * blockDim.x + 1;
  int y = threadIdx.y + blockIdx.y * blockDim.y + 1;

  if(x >= ni-1 || y >= nj-1) return;

  // evaluate derivatives

  #ifndef PITCH
  d2tdx2 = temp_in[I2D(ni, x - 1, y)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x + 1, y)];
  d2tdy2 = temp_in[I2D(ni, x, y - 1)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x, y + 1)];

  // update temperatures
  temp_out[I2D(ni, x, y)] = temp_in[I2D(ni, x, y)]+ fact * (d2tdx2 + d2tdy2);
  #else
  d2tdx2 = temp_in[I2D(pitch1, x - 1, y)] - 2 * temp_in[I2D(pitch1, x, y)] + temp_in[I2D(pitch1, x + 1, y)];
  d2tdy2 = temp_in[I2D(pitch1, x, y - 1)] - 2 * temp_in[I2D(pitch1, x, y)] + temp_in[I2D(pitch1, x, y + 1)];

  // update temperatures
  temp_out[I2D(pitch2, x, y)] = temp_in[I2D(pitch1, x, y)] + fact * (d2tdx2 + d2tdy2);
  #endif
}
#else
__global__ void step_kernel_mod(int ni, int nj, float fact, float* temp_in, float* temp_out)
{

  /*
    COMMENTS: Remember about coalescence (cudaMallocPitch())
    remeber balancing between L1 and Shared Mem cudaDeviceSetCacheConfig
    __shared__ to access shared memory
    __constant__ to set something on constant memory
  */
  //int i00, im10, ip10, i0m1, i0p1;
  float d2tdx2, d2tdy2;

  __shared__ float sharedMem[(BLOCK_WIDTH + 2) * (BLOCK_HEIGHT + 2)];

  int x = threadIdx.x + blockIdx.x * blockDim.x + 1;
  int y = threadIdx.y + blockIdx.y * blockDim.y + 1;

  if(x >= ni-1 || y >= nj-1) return;

  sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)] = temp_in[I2D(ni, x, y)];

// TODO: put most of this inside the same warp

  if(threadIdx.x == 0) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x, threadIdx.y + 1)] = temp_in[I2D(ni, x - 1, y)];
  else if(threadIdx.x == blockDim.x - 1 || x == ni - 2) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 2, threadIdx.y + 1)] = temp_in[I2D(ni, x + 1, y)];

  if(threadIdx.y == 0) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y)] = temp_in[I2D(ni, x, y - 1)];
  else if(threadIdx.y == blockDim.y - 1 || y == nj - 2) sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 2)] = temp_in[I2D(ni, x, y + 1)];

  __syncthreads();

  // evaluate derivatives
  d2tdx2 = sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x, threadIdx.y + 1)]
    - 2 * sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)]
    + sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 2, threadIdx.y + 1)];

  d2tdy2 = sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y)]
    - 2 * sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)]
    + sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 2)];

    //d2tdx2 = temp_in[I2D(ni, x - 1, y)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x + 1, y)];
    //d2tdy2 = temp_in[I2D(ni, x, y - 1)] - 2 * temp_in[I2D(ni, x, y)] + temp_in[I2D(ni, x, y + 1)];

  // update temperatures
  temp_out[I2D(ni, x, y)] = sharedMem[I2D(BLOCK_WIDTH + 2, threadIdx.x + 1, threadIdx.y + 1)] + fact * (d2tdx2 + d2tdy2);

}
#endif
#endif

void step_kernel_ref(int ni, int nj, float fact, float* temp_in, float* temp_out)
{
  int i00, im10, ip10, i0m1, i0p1;
  float d2tdx2, d2tdy2;


  // loop over all points in domain (except boundary)
  for ( int j=1; j < nj-1; j++ ) {
    for ( int i=1; i < ni-1; i++ ) {
      // find indices into linear memory
      // for central point and neighbours
      i00 = I2D(ni, i, j);
      im10 = I2D(ni, i-1, j);
      ip10 = I2D(ni, i+1, j);
      i0m1 = I2D(ni, i, j-1);
      i0p1 = I2D(ni, i, j+1);

      // evaluate derivatives
      d2tdx2 = temp_in[im10]-2*temp_in[i00]+temp_in[ip10];
      d2tdy2 = temp_in[i0m1]-2*temp_in[i00]+temp_in[i0p1];

      // update temperatures
      temp_out[i00] = temp_in[i00]+fact*(d2tdx2 + d2tdy2);
    }
  }
}

int main()
{
  struct timeval stop, start;
  int istep;
  int nstep = STEPS; // number of time steps

  // Specify our 2D dimensions
  const int ni = SIZE;
  const int nj = SIZE;
  float tfac = 8.418e-5; // thermal diffusivity of silver

  float *temp1_ref, *temp2_ref, *temp1, *temp2, *temp_tmp;

  const int size = ni * nj * sizeof(float);

  temp1_ref = (float*)malloc(size);
  temp2_ref = (float*)malloc(size);
  temp1 = (float*)malloc(size);
  temp2 = (float*)malloc(size);

  // Initialize with random data
  for( int i = 0; i < ni*nj; ++i) {
    temp1_ref[i] = temp2_ref[i] = temp1[i] = temp2[i] = (float)rand()/(float)(RAND_MAX/100.0f);
  }

  gettimeofday(&start, NULL);

#ifndef __NVCC__

  // Execute the CPU-only reference version
  for (istep=0; istep < nstep; istep++) {
    step_kernel_ref(ni, nj, tfac, temp1_ref, temp2_ref);

    // swap the temperature pointers
    temp_tmp = temp1_ref;
    temp1_ref = temp2_ref;
    temp2_ref= temp_tmp;
  }

#else

  float *temp1_dev, *temp2_dev;

#ifdef PITCH
  size_t pitch1;
  cudaMallocPitch((void**) &temp1_dev, &pitch1,nj,ni);
  size_t pitch2;
  cudaMallocPitch((void**) &temp2_dev, &pitch2,nj,ni);

  cudaMemcpy2D(temp1_dev, pitch1,temp1, nj,nj,ni, cudaMemcpyHostToDevice);
  cudaMemcpy2D(temp2_dev, pitch2,temp2, nj,nj,ni, cudaMemcpyHostToDevice);

  dim3 block_size(BLOCK_WIDTH, BLOCK_HEIGHT);
  dim3 grid_size = dim3((nj - 3) / BLOCK_WIDTH + 1, (ni - 3) / BLOCK_HEIGHT + 1);

  size_t pitch_tmp;
  // Execute the modified version using same data
  for (istep=0; istep < nstep; istep++) {
    step_kernel_mod<<<grid_size, block_size>>>(ni, nj, tfac, temp1_dev, temp2_dev, pitch1, pitch2);

    // swap the temperature pointers
    temp_tmp = temp1_dev;
    temp1_dev = temp2_dev;
    temp2_dev = temp_tmp;

    pitch_tmp = pitch1;
    pitch1 = pitch2;
    pitch2 = pitch_tmp;
  }
  cudaMemcpy(temp1, temp1_dev, size, cudaMemcpyDeviceToHost);

  cudaDeviceSynchronize();

#else
  cudaMalloc((void**) &temp1_dev, size);
  cudaMalloc((void**) &temp2_dev, size);

  cudaMemcpy(temp1_dev, temp1, size, cudaMemcpyHostToDevice);
  cudaMemcpy(temp2_dev, temp2, size, cudaMemcpyHostToDevice);

  dim3 block_size(BLOCK_WIDTH, BLOCK_HEIGHT);
  dim3 grid_size = dim3((nj - 3) / BLOCK_WIDTH + 1, (ni - 3) / BLOCK_HEIGHT + 1);

  // Execute the modified version using same data
  for (istep=0; istep < nstep; istep++) {
    step_kernel_mod<<<grid_size, block_size>>>(ni, nj, tfac, temp1_dev, temp2_dev);

    // swap the temperature pointers
    temp_tmp = temp1_dev;
    temp1_dev = temp2_dev;
    temp2_dev = temp_tmp;
  }
  cudaMemcpy(temp1, temp1_dev, size, cudaMemcpyDeviceToHost);

  cudaDeviceSynchronize();
#endif

#endif

  gettimeofday(&stop, NULL);
  printf("took %lf s\n", ((stop.tv_sec - start.tv_sec) * 1000000 + stop.tv_usec - start.tv_usec)/1000000.0);

  // float maxError = 0;
  // // Output should always be stored in the temp1 and temp1_ref at this point
  // for( int i = 0; i < ni*nj; ++i ) {
  //   if (abs(temp1[i]-temp1_ref[i]) > maxError) { maxError = abs(temp1[i]-temp1_ref[i]); }
  // }
  //
  // // Check and see if our maxError is greater than an error bound
  // if (maxError > 0.0005f)
  //   printf("Problem! The Max Error of %.5f is NOT within acceptable bounds.\\n", maxError);
  // else
  //   printf("The Max Error of %.5f is within acceptable bounds.\\n", maxError);

  free( temp1_ref );
  free( temp2_ref );
  free( temp1 );
  free( temp2 );

  return 0;
}

'''
with open('assignment-less-dumb.c', 'w') as f:
  f.write(code)


In [84]:
!nvcc assignment-less-dumb.cu -o assignment-less-dumb -DSTEPS=5000 -DSHARED -DSIZE=10000 -DBLOCK_SIZE=16 -DPITCH && /content/assignment-less-dumb

took 30.195786 s
